In [ ]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import torch
import random
import numpy as np

# Set seed for reproducibility
seed_value = 42  # You can change this value to any integer
torch.manual_seed(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)

# If you are using a GPU, set the seed for CUDA as well
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed_value)

## Datasets

In [ ]:
# Load the IMDB dataset
from datasets import load_dataset
dataset = load_dataset("imdb")
test_data = dataset["test"]

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
test_data

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [ ]:
X_test = test_data["text"]

In [ ]:
y_true = test_data["label"]

## Evaluate functions

In [ ]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
from transformers import pipeline
from tqdm import tqdm

def predict(test, model_name, adapter, batch_size=16, tokenizer_name=None):
    y_pred = []

    # Create the pipeline based on whether tokenizer_name is provided
    if tokenizer_name:
        tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        pipe = pipeline(task="text-classification", model=model_name, tokenizer=tokenizer, framework="pt", device=0)
    else:
        pipe = pipeline(task="text-classification", model=model_name, framework="pt", device=0)

    for i in tqdm(range(0, len(test), batch_size)):
        batch = test[i:i+batch_size]
        results = pipe(batch, truncation=True)

        for result in results:
            standardized_label = adapter(result)
            y_pred.append(standardized_label)

    return y_pred


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from collections import Counter

def evaluate_binary(y_true, y_pred):
    # Use binary mapping
    mapping = {'positive': 1, 'negative': 0}

    # Convert y_true and y_pred based on binary mapping
    y_true_binary = [0 if label == 0 else 1 for label in y_true]  # IMDB labels are 0 and 1
    y_pred_binary = [mapping[label] if label in mapping else 0 for label in y_pred]

    # Calculate overall accuracy
    accuracy = accuracy_score(y_true=y_true_binary, y_pred=y_pred_binary)
    print(f"Binary Accuracy: {accuracy:.3f}")

    # Generate binary classification report
    print("\nBinary Classification Report:")
    print(classification_report(y_true=y_true_binary, y_pred=y_pred_binary, target_names=['negative', 'positive']))

    # Generate binary confusion matrix
    print("\nBinary Confusion Matrix:")
    print(confusion_matrix(y_true=y_true_binary, y_pred=y_pred_binary))




## Adapter functions

In [ ]:
# Adapter for ('LABEL_0'/'LABEL_1')
def adapter_model_numeric(output):
    if output['label'] == 'LABEL_1':
        return 'positive'
    elif output['label'] == 'LABEL_0':
        return 'negative'
    else:
        return 'none'

# Adapter for ('POSITIVE'/'NEGATIVE'/'NEUTRAL')
def adapter_model_text(output):
    label = output['label'].lower()
    if "positive" in label:
        return 'positive'
    elif "negative" in label:
        return 'negative'
    elif "neutral" in label:
        return 'neutral'
    else:
        return 'none'

## Models

In [ ]:
baseline_model_name = "distilbert-base-uncased"
benchmark_model_name = "distilbert-base-uncased-finetuned-sst-2-english"
sft_model_name = "wsklee/distilbert-sentiment-imdb-sft"

### Evaluation: Baseline DistilBERT model

In [ ]:
# Call the predict function with test data
y_pred = predict(X_test, baseline_model_name, adapter_model_numeric)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

100%|██████████| 1563/1563 [04:21<00:00,  5.99it/s]


In [ ]:
# Call the simplified evaluate function for binary classification
evaluate_binary(y_true, y_pred)

Binary Accuracy: 0.406

Binary Classification Report:
              precision    recall  f1-score   support

    negative       0.39      0.32      0.35     12500
    positive       0.42      0.49      0.45     12500

    accuracy                           0.41     25000
   macro avg       0.40      0.41      0.40     25000
weighted avg       0.40      0.41      0.40     25000


Binary Confusion Matrix:
[[3977 8523]
 [6336 6164]]


In [ ]:
# Print samples of predictions and true labels for verification
print("Sample y_pred:", y_pred[:100])
print("Sample y_true:", y_true[:100])

Sample y_pred: ['negative', 'positive', 'positive', 'positive', 'positive', 'positive', 'negative', 'positive', 'negative', 'positive', 'positive', 'negative', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'negative', 'positive', 'positive', 'positive', 'negative', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'negative', 'positive', 'negative', 'negative', 'positive', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'negative', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'negative', 'positive', 'positive', 'positive', 'positive', 'positive', 'negative', 'positive', 'positive', 'negative', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 

In [ ]:
# Sample inference
from transformers import pipeline

# Load model and tokenizer using the pipeline
classifier = pipeline("text-classification", model=baseline_model_name, device=0)

# Sample texts for inference
texts = [
    "The movie was absolutely fantastic!",
    "I didn't enjoy the film; it was quite boring.",
    "An average experience, nothing special or memorable."
]

# Perform inference
results = classifier(texts)
for text, result in zip(texts, results):
    print(f"Input: {text}")
    print(f"Output: {result}\n")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Input: The movie was absolutely fantastic!
Output: {'label': 'LABEL_0', 'score': 0.5017849206924438}

Input: I didn't enjoy the film; it was quite boring.
Output: {'label': 'LABEL_0', 'score': 0.5064137578010559}

Input: An average experience, nothing special or memorable.
Output: {'label': 'LABEL_0', 'score': 0.5088108777999878}



### Evaluation: distilbert-base-uncased-finetuned-sst-2-english

In [ ]:
# Call the predict function with test data
y_pred = predict(X_test, benchmark_model_name, adapter_model_text)

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

100%|██████████| 1563/1563 [04:26<00:00,  5.87it/s]


In [ ]:
# Call the simplified evaluate function for binary classification
evaluate_binary(y_true, y_pred)

Binary Accuracy: 0.891

Binary Classification Report:
              precision    recall  f1-score   support

    negative       0.87      0.92      0.89     12500
    positive       0.91      0.86      0.89     12500

    accuracy                           0.89     25000
   macro avg       0.89      0.89      0.89     25000
weighted avg       0.89      0.89      0.89     25000


Binary Confusion Matrix:
[[11494  1006]
 [ 1726 10774]]


### Evaluation: DistilBERT-SFT model

In [ ]:
# Call the predict function with test data
y_pred = predict(X_test, sft_model_name, adapter_model_numeric)

config.json:   0%|          | 0.00/626 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

100%|██████████| 1563/1563 [04:38<00:00,  5.62it/s]


In [ ]:
# Call the simplified evaluate function for binary classification
evaluate_binary(y_true, y_pred)

Binary Accuracy: 0.915

Binary Classification Report:
              precision    recall  f1-score   support

    negative       0.90      0.93      0.92     12500
    positive       0.93      0.90      0.91     12500

    accuracy                           0.92     25000
   macro avg       0.92      0.92      0.92     25000
weighted avg       0.92      0.92      0.92     25000


Binary Confusion Matrix:
[[11656   844]
 [ 1279 11221]]


In [ ]:
# Print samples of predictions and true labels for verification
print("Sample y_pred:", y_pred[:100])
print("Sample y_true:", y_true[:100])

Sample y_pred: ['negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 

In [ ]:
# Sample inference
from transformers import pipeline

# Load model and tokenizer using the pipeline
classifier = pipeline("text-classification", model=sft_model_name, device=0)

# Sample texts for inference
texts = [
    "The movie was absolutely fantastic!",
    "I didn't enjoy the film; it was quite boring.",
    "An average experience, nothing special or memorable."
]

# Perform inference
results = classifier(texts)
for text, result in zip(texts, results):
    print(f"Input: {text}")
    print(f"Output: {result}\n")

Input: The movie was absolutely fantastic!
Output: {'label': 'LABEL_1', 'score': 0.9981886744499207}

Input: I didn't enjoy the film; it was quite boring.
Output: {'label': 'LABEL_0', 'score': 0.997225821018219}

Input: An average experience, nothing special or memorable.
Output: {'label': 'LABEL_0', 'score': 0.9878718852996826}

